In [172]:
cd /Volumes/LaCie/FPV_GIS/Evaporation/stations/az

/Volumes/LaCie/FPV_GIS/Evaporation/stations/az


In [173]:
ls

Az020060.par*    Az022140.par*    Az025635.par*    Az028499.par*
Az020080.par*    Az022664.par*    Az025700.par*    Az028619.par*
Az020159.par*    Az022754.par*    Az025924.par*    Az028796.par*
Az020750.par*    Az022807.par*    Az026119.par*    Az028800.par*
Az020808.par*    Az023010.par*    Az026132.par*    Az028815.par*
Az020949.par*    Az023027.par*    Az026250.par*    Az028820.par*
Az020958.par*    Az023160.par*    Az026323.par*    Az028865.par*
Az021001.par*    Az023303.par*    Az026468.par*    Az028895.par*
Az021026.par*    Az023393.par*    Az026481.par*    Az029114.par*
Az021231.par*    Az023961.par*    Az026796.par*    Az029158.par*
Az021282.par*    Az024089.par*    Az027281.par*    Az029271.par*
Az021306.par*    Az024418.par*    Az027390.par*    Az029287.par*
Az021314.par*    Az024453.par*    Az027435.par*    Az029309.par*
Az021330.par*    Az024586.par*    Az027480.par*    Az029334.par*
Az021353.par*    Az024675.par*    Az027593.par*    Az029359.par*
Az021514.par*    Az024686

In [174]:
import numpy as np
import os
import pandas as pd

In [176]:
with open ("Az020060.par", "r") as myfile:
    data=myfile.readlines()
data

[' AGUILA AZ                                2  60 0\n',
 ' LATT=  33.95 LONG=-113.18 YEARS= 64. TYPE= 3\n',
 ' ELEVATION = 2180. TP5 = 1.65 TP6= 3.12\n',
 ' MEAN P    .28   .33   .30   .23   .21   .14   .24   .35   .35   .31   .30   .30\n',
 ' S DEV P   .34   .34   .42   .29   .24   .14   .28   .46   .47   .44   .38   .34\n',
 ' SKEW  P  2.55  1.67  3.19  3.43  1.87  2.04  2.27  3.13  2.63  3.35  2.85  2.43\n',
 ' P(W/W)    .36   .39   .33   .37   .18   .21   .26   .27   .30   .22   .27   .35\n',
 ' P(W/D)    .07   .07   .07   .04   .01   .02   .09   .11   .05   .05   .05   .07\n',
 ' TMAX AV 62.38 65.82 70.70 80.27 88.98 98.91102.64100.26 95.33 85.08 71.84 63.77\n',
 ' TMIN AV 32.98 35.45 38.71 44.47 51.65 60.55 70.42 69.51 62.30 50.25 38.66 33.36\n',
 ' SD TMAX  8.12  8.47  8.46  8.87  8.07  6.81  5.21  5.29  6.76  8.46  8.56  7.91\n',
 ' SD TMIN  6.71  6.92  6.29  6.64  6.49  7.18  5.85  5.86  6.91  7.19  6.92  6.52\n',
 ' SOL.RAD  286.  393.  517.  623.  696.  732.  661.  617.  560

In [177]:
lat, lon = float(data[1][7:14]), float(data[1][19:27])

In [193]:
lat, lon

(33.95, -113.18)

In [194]:
Tmax = np.array(data[8][8:-1].split()).astype(float)
Tmin = np.array(data[9][8:-1].split()).astype(float)
T = (Tmax + Tmin) / 2 # Average Temperature [C]
T = (T - 32) * (5.0/9.0) # Average Temperature [C]
Tdew = np.array(data[15][8:-1].split()).astype(float) # Dew point temperature (C)
Tdew = (Tdew - 32) * (5.0/9.0) # Dew point temperature [C]

ValueError: could not convert string to float: '98.91102.64100.26'

In [195]:
T, Tdew

(array([  6.26388889,   8.88888889,  13.21666667,  18.41388889,
         22.56111111,  26.725     ,  28.65555556,  28.26388889,
         24.34444444,  18.88888889,  12.10833333,   7.71111111]),
 array([ -1.66666667,   0.        ,   0.55555556,   7.22222222,
         13.33333333,  16.66666667,  17.22222222,  16.11111111,
         14.44444444,  10.        ,   2.77777778,  -0.55555556]))

In [196]:
delta = ( 4098 * (0.6108 * np.exp( (17.27 * T) / (T + 237.3) ) ) ) / ( (T + 237.3)**2 ) # slope of vapor pressure-temperature curve [kPa/C]
G = 0 # Soil heat flux ( assume zero for bodies of water ) [MJ/m2/day]
gamma = 0.067 # psychometric constant [kPa/C]
Rn = np.array(data[12][8:-1].split()).astype(float) # Solar Radiation (langleys/day)
Rn = Rn * 0.04184 # Solar Radiation [MJ/m2/day]
Rn = Rn * (1-0.27)

In [197]:
G, gamma

(0, 0.067)

In [198]:
delta, Rn

(array([ 0.06578613,  0.07704394,  0.09919597,  0.1327561 ,  0.16602007,
         0.20623807,  0.22749607,  0.22304068,  0.18234868,  0.13625342,
         0.09306228,  0.07180447]),
 array([  8.7353552,  12.0034776,  15.7908344,  19.0284136,  21.2580672,
         22.3576224,  20.1890552,  18.8451544,  17.104192 ,  13.3473784,
         10.1708856,   8.2772072]))

In [199]:
time_pk_ind = np.linspace(17,77,16).astype(int) # indices of wind direction percent times
mean_wind_ind = time_pk_ind + 1 # indices of mean wind speeds

wind_dir_percents = np.empty(shape=(17,12)) # initialize array (16 wind directions + 1 calm X 12 months )
wind_speeds = np.empty(shape=(17,12)) # initialize array (16 wind directions + 1 calm X 12 months )

for i in range(0,16): # populate arrays from data
    wind_dir_percents[i] = np.array(data[time_pk_ind[i]][8:-1].split()).astype(float)
    wind_speeds[i] = np.array(data[mean_wind_ind[i]][8:-1].split()).astype(float)
wind_dir_percents[16] = np.array(data[81][8:-1].split()).astype(float)
wind_speeds[16] = np.zeros(shape=(1,12)) # account for calm conditions

percent_time_wind = wind_dir_percents.sum(axis=0) # Check: Must add to 100 % in each month

u2 = (wind_speeds * (wind_dir_percents/100)).sum(axis=0)*0.44704 # weighted average of all 16 wind directions [m/s]

In [200]:
u2.mean()

1.4441854817866668

In [201]:
es = 0.611 * ( 10 ** ( (7.5*T) / (237.3 + T) ) ) # saturated vapor pressure (kPa)
ea = 0.611 * ( 10 ** ( (7.5*Tdew) / (237.3 + Tdew) ) ) # actual vapor pressure (kPa)

In [202]:
es, ea

(array([ 0.9526284 ,  1.13981926,  1.51957964,  2.11891828,  2.73646771,
         3.50915276,  3.92765763,  3.83940627,  3.04699102,  2.18282347,
         1.41303507,  1.05216761]),
 array([ 0.54074522,  0.611     ,  0.63614899,  1.01756377,  1.53119477,
         1.89771848,  1.96574873,  1.8317597 ,  1.64578005,  1.22833431,
         0.74613711,  0.58673414]))

In [203]:
ETo = ( 0.408 * delta * (Rn - G) + gamma * ( 900 / (T + 273) ) * u2 * (es - ea) ) / (delta + gamma * (1 + 0.34 * u2)) # water volume evapotranspiration (mm/day)
ETo = (ETo * 0.00328084) # (ft/day)
num_days_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
Annual_Evap = (ETo * num_days_in_month).sum() # should be about 7 feet per year

In [204]:
Annual_Evap # ft/yr

6.0322100464222128

In [205]:
Evaporation_Data.loc[len(Evaporation_Data)] = [lat, lon, Annual_Evap]

In [206]:
Evaporation_Data

,lat,lon,ann_evap_ft
0,48.33,-96.78,4.024998
1,48.33,-96.78,3.238978
2,37.78,-81.18,4.498008
3,37.78,-81.18,3.448900
4,42.17,-119.90,5.465522
5,42.17,-119.90,4.318932
6,29.73,-84.98,6.075280
7,29.73,-84.98,4.666703
8,32.43,-99.68,7.267623
9,32.43,-99.68,5.886043


In [210]:
4.024998-3.238978, 4.498008-3.448900, 5.465522-4.318932, 6.075280-4.666703, 7.267623-5.886043, 7.683970-6.032210

(0.7860200000000002,
 1.0491079999999995,
 1.1465899999999998,
 1.4085770000000002,
 1.3815800000000005,
 1.6517600000000003)

In [217]:
np.array([(4.024998-3.238978)/4.024998, (4.498008-3.448900)/4.498008, (5.465522-4.318932)/5.465522, (6.075280-4.666703)/6.075280, (7.267623-5.886043)/7.267623, (7.683970-6.032210)/7.683970]) * 100 / 27

array([ 0.72327618,  0.86384577,  0.77698519,  0.85871793,  0.70407655,
        0.7961548 ])